In [343]:
import vectorbt as vbt
data_exchange = 'binanceusdm'
data_timeframe = '1h'
data_start = '2023-01-01'
data_end = '2023-08-31'
bband_sma = 20
bband_std = 2
bband_bandwidth_thr = 0.015 # max 0.22

In [344]:
btc_price = vbt.CCXTData.download_symbol('BTC/USDT', exchange=data_exchange, timeframe=data_timeframe, start=data_start, end=data_end)
eth_price = vbt.CCXTData.download_symbol('ETH/USDT', exchange=data_exchange, timeframe=data_timeframe, start=data_start, end=data_end)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [345]:
btc_price['btc_eth'] = btc_price['Close'] / eth_price['Close']
bband = vbt.BBANDS.run(btc_price['btc_eth'], window=bband_sma, alpha=bband_std)
btc_price['bandwidth'] = bband.bandwidth
btc_price['bandwidth'].fillna(0.0, inplace=True)
bband.plot()

FigureWidget({
    'data': [{'line': {'color': 'rgba(127,127,127,0.7500)'},
              'name': 'Lower Band',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6c6cc2f6-fc30-4437-94f1-373a19c3a620',
              'x': array([datetime.datetime(2022, 12, 31, 16, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 17, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 18, 0, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2023, 8, 30, 13, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 14, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 15, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([        nan,         nan,         nan, ..., 15.88710801, 15.89669178,
                   

In [346]:
bband.bandwidth[bband.bandwidth > bband_bandwidth_thr]
bband.bandwidth.mean()

0.01414273015840736

In [347]:
# long BTC short ETH, close at middle
btc_price['upper_crossed_above'] = bband.upper_crossed_above(btc_price['btc_eth']) & (btc_price['bandwidth'] > bband_bandwidth_thr)
btc_price['middle_crossed_below'] = bband.middle_crossed_below(btc_price['btc_eth'])

In [348]:
entries = btc_price['upper_crossed_above']
exits = btc_price['middle_crossed_below']
pf = vbt.Portfolio.from_signals(btc_price['Close'], entries, exits, max_size=1, init_cash=100000)
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '4140d187-92cd-4169-bb6f-33d9ddf59405',
              'x': array([datetime.datetime(2022, 12, 31, 16, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 17, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 18, 0, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2023, 8, 30, 13, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 14, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 15, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([16596. , 16573.4, 16571.7, ..., 27363.7, 27096.1,

In [349]:
pf.stats()

Start                         2022-12-31 16:00:00+00:00
End                           2023-08-30 15:00:00+00:00
Period                                242 days 00:00:00
Start Value                                    100000.0
End Value                                      106596.7
Total Return [%]                                 6.5967
Benchmark Return [%]                           63.84671
Max Gross Exposure [%]                        29.172944
Total Fees Paid                                     0.0
Max Drawdown [%]                               2.482099
Max Drawdown Duration                 124 days 21:00:00
Total Trades                                         49
Total Closed Trades                                  49
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  59.183673
Best Trade [%]                                 10.14753
Worst Trade [%]                               -4

In [350]:
pf = vbt.Portfolio.from_signals(eth_price['Close'], short_entries=entries, short_exits=exits, size=btc_price['btc_eth'], init_cash=100000)
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'acafd908-32cf-49c9-bfa3-8574ba06208b',
              'x': array([datetime.datetime(2022, 12, 31, 16, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 17, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 18, 0, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2023, 8, 30, 13, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 14, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 15, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1202.49, 1200.59, 1199.59, ..., 1717.62, 1702.12,

In [351]:
pf.stats()

Start                         2022-12-31 16:00:00+00:00
End                           2023-08-30 15:00:00+00:00
Period                                242 days 00:00:00
Start Value                                    100000.0
End Value                                  92938.777384
Total Return [%]                              -7.061223
Benchmark Return [%]                          41.613652
Max Gross Exposure [%]                              0.0
Total Fees Paid                                     0.0
Max Drawdown [%]                                8.86361
Max Drawdown Duration                 229 days 02:00:00
Total Trades                                         49
Total Closed Trades                                  49
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  40.816327
Best Trade [%]                                 6.822715
Worst Trade [%]                              -10

In [352]:
# long ETH short BTC, close at middle
btc_price['lower_crossed_below'] = bband.lower_crossed_below(btc_price['btc_eth']) & (btc_price['bandwidth'] > bband_bandwidth_thr)
btc_price['middle_crossed_above'] = bband.middle_crossed_above(btc_price['btc_eth'])

In [353]:
entries = btc_price['lower_crossed_below']
exits = btc_price['middle_crossed_above']
pf = vbt.Portfolio.from_signals(eth_price['Close'], entries, exits, size=btc_price['btc_eth'], init_cash=100000)
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '06c71c83-e2cc-4c84-a2ce-654934bc6ba9',
              'x': array([datetime.datetime(2022, 12, 31, 16, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 17, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 18, 0, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2023, 8, 30, 13, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 14, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 15, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1202.49, 1200.59, 1199.59, ..., 1717.62, 1702.12,

In [354]:
pf.stats()

Start                         2022-12-31 16:00:00+00:00
End                           2023-08-30 15:00:00+00:00
Period                                242 days 00:00:00
Start Value                                    100000.0
End Value                                 103114.675182
Total Return [%]                               3.114675
Benchmark Return [%]                          41.613652
Max Gross Exposure [%]                         31.03942
Total Fees Paid                                     0.0
Max Drawdown [%]                               3.968904
Max Drawdown Duration                 138 days 09:00:00
Total Trades                                         41
Total Closed Trades                                  41
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  46.341463
Best Trade [%]                                12.235572
Worst Trade [%]                               -5

In [355]:
pf = vbt.Portfolio.from_signals(btc_price['Close'], short_entries=entries, short_exits=exits, max_size=1, init_cash=100000)
pf.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '76b5b44e-4ed4-4827-898c-200b9a75dc6e',
              'x': array([datetime.datetime(2022, 12, 31, 16, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 17, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 12, 31, 18, 0, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2023, 8, 30, 13, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 14, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 8, 30, 15, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([16596. , 16573.4, 16571.7, ..., 27363.7, 27096.1,

In [356]:
pf.stats()

Start                         2022-12-31 16:00:00+00:00
End                           2023-08-30 15:00:00+00:00
Period                                242 days 00:00:00
Start Value                                    100000.0
End Value                                       95441.2
Total Return [%]                                -4.5588
Benchmark Return [%]                           63.84671
Max Gross Exposure [%]                              0.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               7.118069
Max Drawdown Duration                 230 days 01:00:00
Total Trades                                         41
Total Closed Trades                                  41
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  53.658537
Best Trade [%]                                 4.904259
Worst Trade [%]                               -1